In [1]:
using Plots

include("ESPDESolve.jl")

$$
\hat{f}=r^2\left[4(|k| \hat{h})*(|k| \hat{h})+8|k|\left[\hat{h}*(|k| \hat{h})\right]-8 (k^2 \hat{h})*\hat{h}-4(k \hat{h})*(k \hat{h})\right],
$$

In [2]:
N = 50
M = 30
L = 2π
Δts = [0.005;0.005;0.001]
rs = [1.5;3.8;5.]
h₀(x) = 0.01cos(x)
ĥ⁰ = rfftemplate(N)
ĥ⁰[1+1] = 0.01/2

hs = Dict()
ĥs = Dict()
ts = Dict()
for (r,Δt) in zip(rs,Δts)
    t,ĥ = solve(ĥ⁰,N;L=L,r=r,Δt=Δt,M=M)
    ĥs[r] = ĥ
    ts[r] = t
    hs[r] = zeros(Float64,N,M+1)
    for t in 1:M+1
       hs[r][:,t] = irfft(ĥ[:,t],N) 
    end
end

In [5]:
plotly()
x = range(0,2π,length=N+1)
for r in rs
    p = plot(size = (600, 400), title="r=$r")
    t = ts[r]
    h = hs[r]
    for i in range(1,stop=M,step=floor(Int,M/7))
        plot!(p, x, [h[:,i];h[1,i]] ,label="t=$(t[i])")
    end
    xlabel!(p,"x")
    ylabel!(p,"h")
    idxs = Vector(0:0.5:2)
    xticks!(p,(π * idxs,string.(idxs).* " π"))
    display(p)
    # savefig(p,"../img/surface_snapshot(r=$r).pdf")
end

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyKaleido not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyKaleido")` to install the PlotlyKaleido package.
└ @ Plots ~/.julia/packages/Plots/QZRtR/src/backends.jl:552


In [6]:
plotly()
for r in rs
    p = surface(hs[r],size=(600,500))
    xlabel!(p,"t")
    tidxs = Vector(range(1,M+1,step=5))
    xticks!(p,(tidxs,string.(ts[r][tidxs])))
    idxs = Vector(range(0,N,length=5))
    yticks!(p,(idxs,string.(2*idxs/(N)).* " π"))
    ylabel!(p,"x")
    zlabel!(p,"h")
    title!(p,"r=$r")
    display(p)
    # savefig(p,"../img/surface_3d(r=$r).pdf")
end